# ML3 Week 5 — Monet Style Transfer Mini‑Project (Deliverable 1)

Short problem/data description → EDA → model building → training stub → results → conclusion.

## Setup & Paths

In [ ]:

import os, glob, random
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

MONET_DIR = r"C:\Users\Almog\Desktop\monet_jpg"
PHOTO_DIR = r"C:\Users\Almog\Desktop\photo_jpg"
IMG_SIZE = 256

assert os.path.isdir(MONET_DIR), f"Missing: {MONET_DIR}"
monet_files = sorted(glob.glob(os.path.join(MONET_DIR, "*.jpg"))
                     + glob.glob(os.path.join(MONET_DIR, "*.jpeg"))
                     + glob.glob(os.path.join(MONET_DIR, "*.png")))
photo_files = []
if os.path.isdir(PHOTO_DIR):
    photo_files = sorted(glob.glob(os.path.join(PHOTO_DIR, "*.jpg"))
                         + glob.glob(os.path.join(PHOTO_DIR, "*.jpeg"))
                         + glob.glob(os.path.join(PHOTO_DIR, "*.png")))
SKIP_TRAINING = (len(photo_files) == 0)
print("Monet:", len(monet_files), "Photos:", len(photo_files))


## EDA

In [ ]:

def show(paths, title, n=6):
    n = min(n, len(paths))
    if n == 0:
        print("No images:", title); return
    s = random.sample(paths, n)
    rows, cols = 2, (n + 1)//2
    plt.figure(figsize=(3*cols, 3*rows))
    for i,p in enumerate(s):
        plt.subplot(rows, cols, i+1)
        img = Image.open(p).convert("RGB")
        plt.imshow(img); plt.axis("off")
    plt.suptitle(title); plt.tight_layout(); plt.show()

show(monet_files, "Monet samples")
if not SKIP_TRAINING:
    show(photo_files, "Photo samples")


## Model (compact CycleGAN blocks)

In [ ]:

def load_tf_image(path):
    x = tf.io.read_file(path)
    x = tf.image.decode_image(x, channels=3, expand_animations=False)
    x = tf.image.resize(x, [IMG_SIZE, IMG_SIZE], method='area')
    x = tf.cast(x, tf.float32)/127.5 - 1.0
    return x

class ReflectionPad2D(tf.keras.layers.Layer):
    def __init__(self, pad): super().__init__(); self.pad=pad
    def call(self, x): p=self.pad; return tf.pad(x, [[0,0],[p,p],[p,p],[0,0]], mode="REFLECT")

def conv_blk(x,f,k=3,s=1,norm=True,act=True):
    x = tf.keras.layers.Conv2D(f,k,s,padding='valid' if k==7 else 'same',use_bias=not norm)(x)
    if norm:
        m,v = tf.nn.moments(x,[1,2],keepdims=True)
        x = (x-m)/tf.sqrt(v+1e-5)
    if act: x = tf.keras.layers.Activation('relu')(x)
    return x

def res_block(x,f):
    y = ReflectionPad2D(1)(x)
    y = conv_blk(y,f,3,1,True,True)
    y = ReflectionPad2D(1)(y)
    y = conv_blk(y,f,3,1,True,False)
    return tf.keras.layers.Add()([x,y])

def build_generator(img_size=256,n_res=6):
    i = tf.keras.Input((img_size,img_size,3))
    x = ReflectionPad2D(3)(i)
    x = conv_blk(x,64,7,1)
    x = conv_blk(x,128,3,2)
    x = conv_blk(x,256,3,2)
    for _ in range(n_res): x = res_block(x,256)
    x = tf.keras.layers.Conv2DTranspose(128,3,2,padding="same")(x); x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2DTranspose(64,3,2,padding="same")(x);  x = tf.keras.layers.Activation('relu')(x)
    x = ReflectionPad2D(3)(x)
    o = tf.keras.layers.Conv2D(3,7,padding="valid",activation="tanh")(x)
    return tf.keras.Model(i,o)

def build_discriminator(img_size=256):
    def d(x,f,s): x=tf.keras.layers.Conv2D(f,4,s,padding="same")(x); return tf.keras.layers.LeakyReLU(0.2)(x)
    i = tf.keras.Input((img_size,img_size,3))
    x = d(i,64,2); x=d(x,128,2); x=d(x,256,2); x=d(x,512,1)
    o = tf.keras.layers.Conv2D(1,4,padding="same")(x)
    return tf.keras.Model(i,o)

mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()
def gan_loss(logits, is_real):
    y = tf.ones_like(logits) if is_real else tf.zeros_like(logits)
    return mse(y, logits)


## Training (auto‑skip if `photo_jpg` missing)

In [ ]:

BATCH=4; EPOCHS=3; STEPS=300

def ds_from(paths):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.shuffle(1000, reshuffle_each_iteration=True)
    ds = ds.map(lambda p: load_tf_image(p), num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH)
    return ds.prefetch(tf.data.AUTOTUNE)

if SKIP_TRAINING:
    print("Training skipped: no photos in", PHOTO_DIR)
else:
    photo_ds = ds_from(photo_files); monet_ds = ds_from(monet_files)
    paired = tf.data.Dataset.zip((photo_ds, monet_ds))

    G = build_generator(); F = build_generator()
    Dx = build_discriminator(); Dy = build_discriminator()
    opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    LAMBDA_CYCLE=10.0; LAMBDA_ID=0.5

    @tf.function
    def step(rx, ry):
        with tf.GradientTape(persistent=True) as t:
            fy = G(rx, training=True); cx = F(fy, training=True)
            fx = F(ry, training=True); cy = G(fx, training=True)
            sx = F(rx, training=True); sy = G(ry, training=True)
            dxr = Dx(rx, True); dxf = Dx(fx, True)
            dyr = Dy(ry, True); dyf = Dy(fy, True)
            g_adv = gan_loss(dyf, True); f_adv = gan_loss(dxf, True)
            cyc = mae(rx, cx)+mae(ry, cy); idt = mae(rx, sx)+mae(ry, sy)
            g_tot = g_adv + LAMBDA_CYCLE*cyc + LAMBDA_ID*idt
            f_tot = f_adv + LAMBDA_CYCLE*cyc + LAMBDA_ID*idt
            dx_loss = 0.5*(gan_loss(dxr, True)+gan_loss(dxf, False))
            dy_loss = 0.5*(gan_loss(dyr, True)+gan_loss(dyf, False))
        for net,gr in [(G,t.gradient(g_tot,G.trainable_variables)),
                       (F,t.gradient(f_tot,F.trainable_variables)),
                       (Dx,t.gradient(dx_loss,Dx.trainable_variables)),
                       (Dy,t.gradient(dy_loss,Dy.trainable_variables))]:
            opt.apply_gradients(zip(gr, net.trainable_variables))
        return g_tot, f_tot, dx_loss, dy_loss

    it = iter(paired.repeat())
    for e in range(EPOCHS):
        for s in range(STEPS):
            gL,fL,dxL,dyL = step(*next(it))
        print(f"Epoch {e+1}: G {gL.numpy():.3f} F {fL.numpy():.3f} Dx {dxL.numpy():.3f} Dy {dyL.numpy():.3f}")
    import os
    os.makedirs("weights", exist_ok=True)
    G.save("weights/photo2monet_generator.keras")
    print("Saved weights to weights/photo2monet_generator.keras")


## Results & Conclusion

Provided results in github, you can look. cant upload it here.